# Setup

In [ ]:
from ultralytics import YOLO
from ultralytics.models.yolo.obb import OBBTrainer
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt
import torch
import math
%matplotlib inline

# Training

In [ ]:
# Helper for deleting cache files
def del_cache():
    if os.path.exists('../data/MAN/roboflow_oriented_boxes/train/labels.cache'):
        os.remove('../data/MAN/roboflow_oriented_boxes/train/labels.cache')
    if os.path.exists('../data/MAN/roboflow_oriented_boxes/valid/labels.cache'):
        os.remove('../data/MAN/roboflow_oriented_boxes/valid/labels.cache')
del_cache()

## Custom Training

In [ ]:
del_cache() # Deleting cache files if exist

# Custom validator class to override the preprocess method
# class CustomValidator()

# Custom trainer class to override the preprocess_batch method
class CustomTrainer(OBBTrainer):
    def preprocess_batch(self, batch):
        if 'bboxes' in batch and isinstance(batch['bboxes'], torch.Tensor):
            # Set the rotation angle to lock radians in range of 0 to pi/2 (0 to 90 degrees)
            # print(torch.min(batch['bboxes'][:, 4]), torch.max(batch['bboxes'][:, 4]))
            batch['bboxes'][:, 4] = batch['bboxes'][:, 4] % (math.pi / 2)
            # print(torch.min(batch['bboxes'][:, 4]), torch.max(batch['bboxes'][:, 4]))
        else:
            print('Batch does not contain bboxes or is not a tensor.')
        
        # Call the original preprocess_batch method (we do this after modifying the batch as scaling is done in the original method)
        batch = super().preprocess_batch(batch)

        return batch

# Custom configuration for training (required defaults specified)
custom_cfg = {
    'project': '../yolo/runs/obb',
    'name': 'train',
    'device': 'cuda',
    'batch': 16,
    'resume': False,
    'seed': 0,
    'deterministic': True,
    'exist_ok': False,
    'save_period': 1,
    'epochs': 100,
    'model': 'yolo11n-obb.pt',
    'data': '../data/MAN/roboflow_oriented_boxes/data.yaml',
    'task': 'obb',
    'freeze': None,
    'amp': True,
    'imgsz': 640,
    'rect': False,
    'cache': False,
    'single_cls': False,
    'classes': None,
    'fraction': 1.0,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'copy_paste_mode': 'flip',
    'copy_paste': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mask_ratio': 4,
    'overlap_mask': True,
    'bgr': 0.0,
    'workers': 8,
    'mosaic': 1.0,
    'mixup': 0.0,
    'plots': True,
    'nbs': 64,
    'weight_decay': 0.0005,
    'optimizer': 'auto',
    'lr0': 0.01,
    'momentum': 0.937,
    'cos_lr': False,
    'lrf': 0.01,
    'patience': 100,
    'warmup_epochs': 3.0,
    'mode': 'train',
    'time': 24, # Time limit in hours
    'close_mosaic': 10,
    'multi_scale': False,
}

# Create instance of custom trainer
trainer = CustomTrainer(overrides=custom_cfg)

trainer.train() # Train the trainer

del_cache() # Deleting cache files if exist

## Original Training

In [ ]:
model = YOLO(f'yolo11n-obb.pt') # Pretrained model

del_cache() # Deleting cache files if exist

results = model.train(
    data='../data/MAN/roboflow_oriented_boxes/data.yaml',
    time=24,                              # Max training time in hours
    imgsz=640,                            # Image size for training (default for YOLO is 640x640)
    patience=10,                          # Early stopping patience (after this many epochs with no improvement stop training)
    pretrained=False,                     # Don't pre-trained weights (use the latest model)
    plots=True,                           # Create plots

    workers=8,                            # Number of worker threads for data loading
    batch=16,                             # Batch size

    project='../yolo/runs/obb',
    name='train',
)

del_cache() # Deleting cache files if exist

# Evaluating

In [ ]:
# Read images
images_names = os.listdir('../data/MAN/roboflow_oriented_boxes/test/images/')
# Get absolute paths
images = [os.path.abspath(os.path.join('C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\data\\MAN\\roboflow_oriented_boxes\\test\\images\\', image)) for image in images_names]

# Load images
images = [Image.open(image) for image in images]

# Rotate images 90 degrees
images = [image.rotate(90) for image in images]

# Save images
for i, image in enumerate(images):
    image.save(f'../data/MAN/roboflow_oriented_boxes/test/images_rot90/{images_names[i]}')

# Rotate images 90 degrees
images = [image.rotate(90) for image in images]

# Save images
for i, image in enumerate(images):
    image.save(f'../data/MAN/roboflow_oriented_boxes/test/images_rot180/{images_names[i]}')

# Rotate images 90 degrees
images = [image.rotate(90) for image in images]

# Save images
for i, image in enumerate(images):
    image.save(f'../data/MAN/roboflow_oriented_boxes/test/images_rot270/{images_names[i]}')

In [ ]:
# train is the default model, train2 is locked to 0-90, train3 is locked to 0
model_to_eval = 'train4'

# Delete all weights except the best one
for file in os.listdir(f'../yolo/runs/obb/{model_to_eval}/weights/'):
    if file != 'best.pt':
        os.remove(os.path.join(f'../yolo/runs/obb/{model_to_eval}/weights/', file))

# Delete results if exist
if os.path.exists(f'../yolo/runs/obb/{model_to_eval}/results'):
    shutil.rmtree(f'../yolo/runs/obb/{model_to_eval}/results')
if os.path.exists(f'../yolo/runs/obb/{model_to_eval}/results_90'):
    shutil.rmtree(f'../yolo/runs/obb/{model_to_eval}/results_90')
if os.path.exists(f'../yolo/runs/obb/{model_to_eval}/results_180'):
    shutil.rmtree(f'../yolo/runs/obb/{model_to_eval}/results_180')
if os.path.exists(f'../yolo/runs/obb/{model_to_eval}/results_270'):
    shutil.rmtree(f'../yolo/runs/obb/{model_to_eval}/results_270')

In [ ]:
model = YOLO(f'../yolo/runs/obb/{model_to_eval}/weights/best.pt') # load best model from training
model.eval() # Set the model to evaluation mode

# Getting absolute paths of test images
images_names = os.listdir('../data/MAN/roboflow_oriented_boxes/test/images/')
# Get absolute paths
images = [os.path.abspath(os.path.join('C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\data\\MAN\\roboflow_oriented_boxes\\test\\images\\', image)) for image in images_names]
images_90 = [os.path.abspath(os.path.join('C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\data\\MAN\\roboflow_oriented_boxes\\test\\images_rot90\\', image)) for image in images_names]
images_180 = [os.path.abspath(os.path.join('C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\data\\MAN\\roboflow_oriented_boxes\\test\\images_rot180\\', image)) for image in images_names]
images_270 = [os.path.abspath(os.path.join('C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\data\\MAN\\roboflow_oriented_boxes\\test\\images_rot270\\', image)) for image in images_names]

# Run batched inference on test images
results = model(images)
results_90 = model(images_90)
results_180 = model(images_180)
results_270 = model(images_270)

In [ ]:
# Making results folders
if not os.path.exists(f'../yolo/runs/obb/{model_to_eval}/results'):
    os.makedirs(f'../yolo/runs/obb/{model_to_eval}/results')
    os.makedirs(f'../yolo/runs/obb/{model_to_eval}/results_90')
    os.makedirs(f'../yolo/runs/obb/{model_to_eval}/results_180')
    os.makedirs(f'../yolo/runs/obb/{model_to_eval}/results_270')

# Process results list
count = 0
max_angle = 0
for i in range(len(results)):
    # Print results for each image
    result = results[i]

    # for xywhr in result.obb.xywhr:
    if result.obb.xywhr is not None and len(result.obb.xywhr) > 0:
        pred_deg = math.degrees(result.obb.xywhr[0][-1].item())
        if pred_deg > max_angle:
            max_angle = pred_deg
        if not (0 <= pred_deg < 90):
            print(f'Angle is not in range of 0 to 90 degrees: {pred_deg}')
            count += 1

    result_90 = results_90[i]
    result_180 = results_180[i]
    result_270 = results_270[i]

    filename = images[i].split('\\')[-1]

    # Save to disk
    result.save(filename=f'../yolo/runs/obb/{model_to_eval}/results/{filename}')
    result_90.save(filename=f'../yolo/runs/obb/{model_to_eval}/results_90/{filename}')
    result_180.save(filename=f'../yolo/runs/obb/{model_to_eval}/results_180/{filename}')
    result_270.save(filename=f'../yolo/runs/obb/{model_to_eval}/results_270/{filename}')

print(count)
print(max_angle)

In [ ]:
# Display results
images_names = os.listdir(f'../yolo/runs/obb/{model_to_eval}/results/')
images = [os.path.abspath(os.path.join(f'C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\yolo\\runs\\obb\\{model_to_eval}\\results', image)) for image in images_names]
images_90 = [os.path.abspath(os.path.join(f'C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\yolo\\runs\\obb\\{model_to_eval}\\results_90', image)) for image in images_names]
images_180 = [os.path.abspath(os.path.join(f'C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\yolo\\runs\\obb\\{model_to_eval}\\results_180', image)) for image in images_names]
images_270 = [os.path.abspath(os.path.join(f'C:\\Users\\aidan\\OneDrive\\Desktop\\itu\\msc\\courses\\sem4\\thesis\\msc-thesis\\yolo\\runs\\obb\\{model_to_eval}\\results_270', image)) for image in images_names]

# Load image quartets
images = [Image.open(image) for image in images]
images_90 = [Image.open(image) for image in images_90]
images_180 = [Image.open(image) for image in images_180]
images_270 = [Image.open(image) for image in images_270]

for i in range(len(images)):
    print(images_names[i].split('\\')[-1])
    img = images[i]
    img_90 = images_90[i]
    img_180 = images_180[i]
    img_270 = images_270[i]

    # Display images in a grid
    fig, axes = plt.subplots(1, 4, figsize=(20, 20))
    axes[0].imshow(img)
    axes[0].axis('off')
    axes[0].set_title('Original')
    axes[1].imshow(img_90)
    axes[1].axis('off')
    axes[1].set_title('90 degrees')
    axes[2].imshow(img_180)
    axes[2].axis('off')
    axes[2].set_title('180 degrees')
    axes[3].imshow(img_270)
    axes[3].axis('off')
    axes[3].set_title('270 degrees')
    plt.show()